In [35]:
from itertools import product

from tqdm.notebook import tqdm

In [36]:
import highspy


def compile_problem(n, m, include_total):
    h = highspy.Highs()
    # h.setOptionValue("time_limit", 0.1)
    h.setOptionValue("presolve", "off")
    # h.setOptionValue("simplex_iteration_limit", 64)
    h.setOptionValue("log_to_console", True)

    vs = {(i, j): h.addVariable(lb=0.0, ub=1.0) for i, j in product(range(n), range(m))}
    for i in range(n):
        h.addConstr(sum(vs[i, j] for j in range(m)) == 1.0)
    for j in range(m):
        h.addConstr(sum(vs[i, j] for i in range(n)) <= 1.0)
    if include_total:
        h.addConstr(sum(vs[i, j] for i in range(n) for j in range(m)) == 1.0)
    h.minimize(sum(vs[i, j] for i in range(n) for j in range(m)))

    return h

In [37]:
n_values = [2**i for i in range(8)]
m_values = [2**i for i in range(8)]
total_values = [True]
count = len(n_values) * len(m_values) * len(total_values)
args_values = list(product(n_values, m_values, total_values))

In [38]:
# for args in tqdm(args_values, total=count):
for args in tqdm(args_values, total=count):
    problem = compile_problem(*args)
    key = "_".join(map(str, args))
    output_path = f"../resources/highspy/{key}.mps"
    result = problem.writeModel(output_path)

  0%|          | 0/64 [00:00<?, ?it/s]

In [39]:
%%timeit
for args in args_values:
    compile_problem(*args)

1.89 s ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
h = highspy.Highs()

In [41]:
%%timeit
compile_problem(100, 100, True)

381 ms ± 9.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
h.passModel(h.getLp())

77.2 μs ± 746 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [43]:
%%timeit
problem = h.readModel("../resources/highspy/91_91_True.mps")

46.6 μs ± 410 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
